# Python Data Analysis Basics Lab — Open Food Facts (France)

**Objective.** Demonstrate fluency with **NumPy/Pandas**, descriptive analysis, and basic **Matplotlib** plots. 
An optional **PCA** challenge closes the TP.

If you don't manage to end it in class, you can continue it at home.
Send the complete TP to angelo.furno@entpe.fr. 
Add your name to the jupyter notebook.


**Dataset.** We will use a reduced France-focused subset of the Open Food Facts dataset (OFF) hosted at:  
`https://people.licit-lyon.eu/furno/courses/2025/class_01/processed_openfoodfacts_fr.csv`  

Once you have completed the download, add the file in a local path: `data/processed_openfoodfacts_fr.csv`



**Skills you will practice**
- Pandas essentials: Series/DataFrame, indexing/slicing, dtypes, creating columns, missing values, groupby/aggregation.
- Descriptive analytics: distributions, crosstabs, category/brand summaries.
- Plotting with Matplotlib: histogram, bar, box, scatter.
- Optional PCA: scaling, explained variance, **correlation circle (PC1–PC2)**, interpreting loadings.

**Gamified scoring.** Each task has XP. At the end, your **grade /20** is computed from XP.

## XP & Grading 
- Part A — Pandas Fundamentals (max **40 XP**)
- Part B — Descriptive Plots & Interpretation (max **25 XP**)
- Part C — Brands & Categories Mini-Projects (max **15 XP**)
- Part D — PCA (optional, max **20 XP**)

**Target = 80 XP** → Grade = `min(20, round(20 * XP / 80, 1))`.  
Extra XP beyond 80 adds buffer, but max grade is **20/20**.

In [ ]:
#DO NOT CHANGE THIS CELL

import os, numpy as np, pandas as pd, matplotlib.pyplot as plt

try:
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
except Exception:
    print("Scikit-learn not found. Please install it with `pip install scikit-learn`.")
    PCA = None

plt.rcParams['figure.figsize'] = (8, 5)

DATA_URL = "https://people.licit-lyon.eu/furno/courses/2025/processed_openfoodfacts_fr.csv"
LOCAL_FALLBACK = "data/processed_openfoodfacts_fr.csv"

# --- XP utilities (lightweight) ---
XP = 0
HINTS = 4       # number of hint tokens
PENALTY = 1     # XP penalty per hint

def award(points, why=""):
    global XP
    XP += int(points)
    print(f"+{points} XP — {why} (total={XP})")

def grade_summary():
    grade = min(20, round(20 * XP / 80, 1))
    print(f"Final XP={XP} → Grade={grade}/20")
    return grade

## Dataset description

This lab uses a reduced subset of the **Open Food Facts (France)** database.

## What is the Open Food Facts dataset?

The **Open Food Facts (OFF) dataset** is a large, open, collaborative database of packaged food products, available online: `https://fr.openfoodfacts.org/decouvrir`.  
It is sometimes called the *“Wikipedia of food”*, since anyone can contribute information by scanning barcodes, uploading ingredients lists, or adding nutrition facts.

**Why is it interesting for us?**
- It is **real, messy data**: not all fields are complete or consistent.
- It covers a **broad variety of products** consumed daily in France and worldwide.
- It allows analysis of **nutrition, brands, categories, and health scores**.
- It is open data (licensed under ODbL), widely used by researchers, NGOs, and consumer apps.

In this lab we use a **France-focused subset**, with selected columns on metadata, nutrients, and scores.  
Your challenge: **explore, clean, and visualize** this data to uncover insights about the food we eat!


Key columns are:

- **Metadata**
  - `code`: unique product identifier (barcode)
  - `product_name`: product’s name
  - `brands`: comma-separated list of brands
  - `categories`: comma-separated categories (hierarchical)
  - `pnns_groups_1`: high-level food group (e.g., Beverages, Milk and dairy products)
  - `pnns_groups_2`: finer-grained food group (e.g., Fruit juices, Sweetened beverages)
  - `countries_tags`: tags for countries where product is sold

- **Nutrients per 100g**
  - `energy-kcal_100g`: energy content (kcal per 100 g)
  - `sugars_100g`: sugars (g per 100 g)
  - `fat_100g`: total fat (g per 100 g)
  - `saturated-fat_100g`: saturated fat (g per 100 g)
  - `salt_100g`: salt (g per 100 g)
  - `fiber_100g`: fiber (g per 100 g)
  - `proteins_100g`: proteins (g per 100 g)

- **Labels and Scores**
  - `nutriscore_grade`: grade (a–e) of the Nutri-Score label (a=best, e=worst)
  - `nutriscore_score`: numeric score used to derive the Nutri-Score grade
  - `nova_group`: NOVA group classification of food processing level (1–4)
  - `environmental_score_grade`: eco-score label (a–e)

- **Popularity (proxy)**
  - `unique_scans_n`: approximate count of how many times a product has been scanned by users (proxy for popularity)

This dataset is **heterogeneous**: not every product has complete nutrition data, and some fields can be missing or noisy.  
The exercises will guide you through inspecting, cleaning, transforming, and visualizing this data.


Your mission is to **describe** the data: inspect, clean, transform, aggregate, visualize, and **explain** what you see.

**Note: To reduce the size of the original dataset, some columns have already been removed!**

# Part A — Pandas Fundamentals (40 XP)

## A1. Load & Inspect (6 XP)

In this exercise you load the dataset, inspect its structure, and quantify missingness.

**Tasks**
### A1.1 Loading (1 XP)
Load the CSV into a DataFrame `df`. 

Hint: Use the `pd.read_csv()` function. 
   - Docs: [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [2]:
# A1.1 — Load the dataset
# Use the local copy bundled with this TP.
csv_path = "processed_openfoodfacts_fr.csv"
df = pd.read_csv(csv_path)
# Keep a copy of the original for reference if needed
df_raw = df.copy()
df.head()

### A1.2 Preview and Shape (1 XP)
Show the first 5 rows with `df.head()` and print the shape `(rows, cols)`.
   - Docs: [DataFrame.head](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html), [DataFrame.shape](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html)

In [3]:
# A1.2 — Preview and shape
print("Shape (rows, cols):", df.shape)
df.head()

### A1.3 Data info (1 XP)
Display column dtypes and run `df.info(memory_usage="deep")` to see non-null counts and memory usage.
   - Docs: [DataFrame.dtypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html), [DataFrame.info](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html)

In [4]:
# A1.3 — Data info
print("Dtypes:")
print(df.dtypes)
print("\nInfo (deep memory usage):")
df.info(memory_usage="deep")

### A1.4 Exploring Categorical Variables (1 XP)
Explore the values of **categorical variables**, such as:
   - `pnns_groups_1` (main food groups),
   - `nutriscore_grade` (nutrition label a–e),
   - `brands` (most frequent brands).  

   Hint: use `unique`() and `.value_counts()` to display the most frequent categories, and check how diverse they are.  
   - Docs: [Series.value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)

In [5]:
# A1.4 — Explore categorical variables
# View distributions of selected categorical columns
cat_cols = ["pnns_groups_1", "nutriscore_grade", "brands"]
for c in cat_cols:
    if c in df.columns:
        print(f"\nValue counts for {c}:")
        print(df[c].value_counts(dropna=False).head(20))

### A1.5. Missing Values (2 XP)
Compute the **missingness ratio** per column, sort it descending, and show the **top 5**.
   - Docs: [DataFrame.isna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html), [Series.sort_values](https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_values.html)

In [6]:
# A1.5 — Missing values
missing_ratio = df.isna().mean().sort_values(ascending=False)
print("Top missingness ratios:")
missing_ratio.head(20)

In [7]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+6 XP — A1: Completed data loading and basic inspection (total=6)


## A2. Selecting, Filtering, Slicing (8 XP)

Build a focused view of the data to practice column selection, row filtering, and slicing.

### A2.1 — Select a column subset into `df_sub` (1 XP)

Create `df_sub` with columns:  
`product_name, brands, pnns_groups_1, sugars_100g, fat_100g, salt_100g, proteins_100g, nutriscore_grade`.

**Hint:** Use `df[cols].copy()` when you plan to transform this subset.

We’ll build a focused subset for this task.  
💡 When creating a working subset you intend to modify, always use `.copy()` to avoid `SettingWithCopyWarning`. This is not necessary if you just need to explore the dataset without any modification.

In [8]:
# A2.1 — Select a column subset into df_sub
cols = [
    "product_name","brands","pnns_groups_1",
    "sugars_100g","fat_100g","salt_100g","proteins_100g",
    "nutriscore_grade"
]
df_sub = df[cols].copy()
df_sub.head()

### A2.2 — Filter by categories with a boolean mask (1 XP)

Keep only rows where `pnns_groups_1` ∈ {Beverages, Sugary snacks, Milk and dairy products}.

**Hint:** Build a mask with `.isin(...)`, then use `.loc[mask, :]`.  
Avoid chained indexing like `df_sub[df_sub[...] ...]` if you will modify later.

In [9]:
# A2.2 — Filter by categories with a boolean mask
keep_groups = {"Beverages", "Sugary snacks", "Milk and dairy products"}
mask = df_sub["pnns_groups_1"].isin(keep_groups)
df_sub = df_sub.loc[mask].copy()
df_sub["pnns_groups_1"].value_counts()

### A2.3 — Positional slicing with `.iloc` (1 XP)

Show rows **10:20** by **position** (0-based).  
<br>Note: `.iloc[start:stop]` is **exclusive** of `stop`.

- Docs: [DataFrame.iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)

In [10]:
# A2.3 — Positional slicing with .iloc
# Show rows 10 to 19 and first 5 columns
df_sub.iloc[10:20, 0:5]

### A2.4 — Label-based selection with `.loc` (2 XP)

Select **two rows by index label** using `.loc[[label1, label2]]`.

1. First, demonstrate selection *by current index labels*.  
2. Then, change the index to `product_name` (**without dropping** the column), and select two products by name.

**Hints:**
- `.loc` expects **labels** which can be integer or not depending on the index.
- After `set_index("product_name", drop=False)`, index labels become product names.  
- If names are duplicated, `.loc[["Coca Cola", "Nutella"]]` returns **multiple rows per name**.

- Docs: [DataFrame.loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)

In [11]:
# A2.4 — Label-based selection with .loc
# Set product_name as the index (without dropping the column)
df_sub = df_sub.set_index("product_name", drop=False)

# Select two specific products by name if they exist
names = list(df_sub.index.dropna().unique())[:2]
df_sub.loc[names, ["brands","pnns_groups_1","sugars_100g","proteins_100g"]]

### A2.5 — Restoring the original RangeIndex (no XP)

If you prefer position-based operations again, reset the index.

In [12]:
# A2.5 — Reset index back to RangeIndex
df_sub = df_sub.reset_index(drop=True)
df_sub.head(3)

### A2.6 — A bit of thinking here (1 XP)
**`.iloc` vs `.loc`**  
- `.iloc` is **position-based** (uses integer positions). Slices are **half-open**: `start` included, `stop` excluded.  
- `.loc` is **label-based** (uses index labels). When slicing by labels, both **endpoints are included**.  
- `.loc` also accepts boolean masks and column label lists; `.iloc` only accepts integer positions.


### A2.7 — Sorting (1 XP)
Sort by `sugars_100g` descending and show top 10

**Hint:** Use `.sort_values(..., ascending=False).head(10)`.  
Prefer `.loc` if you want to select specific columns.

In [13]:
# TODO A2.7
# Fill in the code to perform the tasks described in the lab instructions.
# Add as many cells as you need below this line.

### A2.8 — Safer column selection after filtering when modifying a dataframe (1 XP)

When you filter rows, prefer **boolean masks with `.loc`** over chained indexing.  

- ✅ Safe: 
        `mask = df["pnns_groups_1"] == "Beverages"; df.loc[mask, "sugars_100g"] = 0`  
- ⚠️ Risky: `df[df["pnns_groups_1"] == "Beverages"]["sugars_100g"] = 0`  

Why?  
- `.loc` applies changes directly to the original DataFrame.  
- Chained indexing can return a **temporary view**; updates may not propagate, and you may see a `SettingWithCopyWarning`.

---

**Minimal example: try this code, observe what happens and describe**

```python

    ex_df = pd.DataFrame({
        "pnns_groups_1": ["Beverages", "Snacks", "Beverages"],
        "sugars_100g": [10, 25, 35]
    })
    print("Original df:\n", ex_df, "\n")

    # --- Approach 1: Boolean mask with .loc (safe) ---
    mask = ex_df["pnns_groups_1"] == "Beverages"
    ex_df.loc[mask, "sugars_100g"] = 0   # modify sugars for beverages
    print("After .loc modification:\n", ex_df, "\n")

    # Reset for comparison
    ex_df["sugars_100g"] = [10, 25, 35]

    # --- Approach 2: Chained filtering (risky) ---
    ex_df[ex_df["pnns_groups_1"] == "Beverages"]["sugars_100g"] = 0
    print("After chained filtering modification:\n", ex_df)

In [14]:
# TODO A2.8
# Fill in the code to perform the tasks described in the lab instructions.
# Add as many cells as you need below this line.

Write your conclusions below (bullet points are fine).
- Use a boolean mask with .loc to modify filtered rows; it updates the original DataFrame reliably.
- Avoid chained indexing like df[df[...]]['col'] = ...; it may write to a temporary copy (no effect) and often raises SettingWithCopyWarning.

In [15]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+8 XP — A2: Completed data subsetting and sorting (total=14)


## A3. String operations & New columns (6 XP)

We now go back to working on the full `df` (not on `df_sub`).

The goal is to create new features from existing columns, using string methods and arithmetic operations.

### A3.1. New columns (1 XP)
Create a new column `sugar_to_protein` as the ratio `sugars_100g / (proteins_100g + 1e-6)`.  
   - The `+1e-6` is to avoid division by zero when proteins = 0.

In [16]:
# A3.1 — New column: sugar_to_protein
df["sugar_to_protein"] = df["sugars_100g"] / (df["proteins_100g"] + 1e-6)
df[["sugars_100g","proteins_100g","sugar_to_protein"]].head()

### A3.2. Another one (2 XP)
Create a boolean column `is_sweet` that is `True` if `sugars_100g > 15`.  
   - Docs: [Comparison ops](https://pandas.pydata.org/docs/user_guide/dsintro.html#operations)

In [17]:
# A3.2 — Boolean column is_sweet
df["is_sweet"] = df["sugars_100g"] > 15
df["is_sweet"].value_counts(dropna=False)

### A3.3. String manipulation + column creation (3XP)
Explore the column `brands`. Some rows contain **multiple brands separated by commas** (e.g., `"Nestlé, Ricoré"`).  
   - Find such rows explicitly using a string condition: `df[df["brands"].str.contains(",", na=False)]`.  
   - Display a few examples to see how the data looks.  
   - Question: how could we extract only the *first* brand from such strings?

In [18]:
# A3.3 — Explore multi-brand rows
multi_brand_rows = df[df["brands"].str.contains(",", na=False)]
multi_brand_rows[["product_name","brands"]].head(10)

Create a new column `brand_main` as the **first brand** listed in `brands`, lowercased and stripped of spaces.  
   - Hint: chain string operations like `str.split(",").str[0].str.strip().str.lower()`  
   - Docs: [Series.str](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling)



In [19]:
# A3.3 — Create brand_main (first brand, cleaned)
df["brand_main"] = (
    df["brands"]
    .str.split(",")
    .str[0]
    .str.strip()
    .str.lower()
)
df[["brands","brand_main"]].head(10)

In [20]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+6 XP — A3: Completed boolean masking and filtering (total=20)


## A4. Types, Duplicates & Missing Values (8 XP)

In this section you will:
- remove duplicate products by name,
- identify and handle missing values,
- convert the Nutri-Score grade to an ordered categorical type,
- perform simple imputations (no `groupby` yet).


### A4.1 — Deduplicate by product name (4 XP)

Many products share the same `product_name` (variants / re-entries).  
We want to keep only the *most complete* row per product.

1. **Normalize the product name**  
   Hint: Use `.str.strip().str.lower()` on the `product_name` column to create a helper column, e.g. `name_norm`.

2. **Build a completeness score**  
   Compute the number of non-missing values **per row** over the *core nutrients*:  
   `["sugars_100g", "fat_100g", "salt_100g", "proteins_100g"]`  
   and optionally: `"energy-kcal_100g"`, `"fiber_100g"`.  
   Hint: Use `notna` and `sum` [DataFrame.notna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.notna.html), [DataFrame.sum](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html)

3. **Sort rows by completeness**  
   Place the most complete entries first.  
   📖 [DataFrame.sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)

4. **Drop duplicates**  
   Keep only one row per normalized name using:  
   ```python
   df.drop_duplicates(subset="name_norm", keep="first")

In [21]:
# A4.1 — Deduplicate by product name
# 1) Normalize product name
df["name_norm"] = df["product_name"].astype(str).str.strip().str.lower()

# 2) Completeness score over core nutrients (and optional extras)
core = ["sugars_100g","fat_100g","salt_100g","proteins_100g"]
optional = [c for c in ["energy-kcal_100g","fiber_100g"] if c in df.columns]
score_cols = core + optional
df["_complete_n"] = df[score_cols].notna().sum(axis=1)

# 3) Sort by completeness (desc) then by non-missing count of all cols as tiebreaker
df = df.sort_values(by=["name_norm","_complete_n"], ascending=[True, False])

# 4) Drop duplicates keeping the most complete
df = df.drop_duplicates(subset="name_norm", keep="first").copy()

# Clean helper columns if desired (keep name_norm for later loc operations)
df = df.drop(columns=["_complete_n"])

### A4.2 — Identify missingness (1 XP)

We now check how complete our data is for the **4 core nutrients**:  
`sugars_100g, fat_100g, salt_100g, proteins_100g`.

1. **Column-wise missingness**  
   - Compute the number of missing values **per column** 
   - This tells you which nutrients are more often missing across all products.  
   <br>
   Hints: Use [DataFrame.isna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html), [DataFrame.sum](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html)

2. **Row-wise missingness**  
   - Create a new column `nutr_missing_n` = number of missing nutrients **per row**.  
   - Use `.isna().sum(axis=1)` to count missing values across columns instead of down rows.  
   ```python
   df["nutr_missing_n"] = df[nutr4].isna().sum(axis=1)

In [22]:
# A4.2 — Identify missingness for the core nutrients
nutr4 = ["sugars_100g","fat_100g","salt_100g","proteins_100g"]

# Column-wise missing count and ratio
col_missing = df[nutr4].isna().sum().to_frame("missing_n")
col_missing["missing_pct"] = (col_missing["missing_n"] / len(df)) * 100
print(col_missing.sort_values("missing_pct", ascending=False))

# Row-wise missingness
df["nutr_missing_n"] = df[nutr4].isna().sum(axis=1)
df["nutr_missing_n"].value_counts().sort_index()

### A4.3 — Remove missing values (1 XP)
Drop rows where all 4 core nutrients are missing.
   These rows are not useful for analysis.  

   Hint: use [DataFrame.dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)

Print how many rows have been removed.

In [23]:
# A4.3 — Drop rows where ALL 4 core nutrients are missing
before = len(df)
df = df.dropna(subset=nutr4, how="all").copy()
after = len(df)
print(f"Rows removed where all 4 nutrients missing: {before - after}")

### A4.4 — Simple imputations (1 XP)

Some nutrient values are missing. To prepare a cleaner dataset, we will create **imputed versions** of selected columns while **keeping the originals unchanged**.

1. **Fill missing fiber with 0**  
   - If the column `fiber_100g` exists, create a new column `fiber_filled` where missing values are replaced with `0`.  
   - Hint: use [`Series.fillna`](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) on the proper column.

2. **Median-impute the 4 core nutrients**  
   - For each core nutrient (`sugars_100g`, `fat_100g`, `salt_100g`, `proteins_100g`), create a new column with suffix `_imp`.  
   - Compute the **global median** of each column with `.median()`.  
   - Replace missing values with that median using `.fillna()`.  

   **Example for sugars:**  
   ```python
   median_sugar = df["sugars_100g"].median()
   df["sugars_100g_imp"] = df["sugars_100g"].fillna(median_sugar)

In [24]:
# A4.4 — Simple imputations
# Create imputed copies without overwriting originals
for c in nutr4:
    imp_col = f"{c}_imp"
    median_val = df[c].median(skipna=True)
    df[imp_col] = df[c].fillna(median_val)

# If fiber_100g exists, create a zero-imputed version
if "fiber_100g" in df.columns:
    df["fiber_100g_imp0"] = df["fiber_100g"].fillna(0.0)

df[[c for c in df.columns if c.endswith("_imp")]].head()

### A4.5 — Convert type for Nutri-Score (1 XP)

The column `nutriscore_grade` currently contains letters (`a`–`e`) as **strings**.  
If we keep them as plain strings, sorting/comparisons will just follow alphabetical order, which is not guaranteed to match the intended **nutritional ranking**.

We want to convert it to an **ordered categorical** type, so that Pandas understands the correct order:  
`a < b < c < d < e`.

**Steps to follow:**
1. Import `CategoricalDtype` from `pandas.api.types`.  
   ```python
   from pandas.api.types import CategoricalDtype
   ```
2.	Define the nutritional order:
   ```python
   nutri_order = CategoricalDtype(
      categories=["a", "b", "c", "d", "e"],
      ordered=True
   )
   ```
3. Apply the conversion
   Hint: use .astype(nutri_order) on the appropriate column
4. Verify (explain how you did it)

In [25]:
# A4.5 — Convert nutriscore_grade to ordered categorical and numeric
# Clean string 'nan' to real NaN before conversion
df["nutriscore_grade"] = df["nutriscore_grade"].astype(str).str.strip().str.lower().replace("nan", np.nan)

# Define ordered category
order = ["a","b","c","d","e"]
df["nutriscore_grade"] = pd.Categorical(df["nutriscore_grade"], categories=order, ordered=True)

# Map to numbers (a=1 .. e=5) for easier averaging (nullable integer)
grade_map = {"a":1, "b":2, "c":3, "d":4, "e":5}
df["grade_num"] = df["nutriscore_grade"].astype(str).map(grade_map).astype("Int64")
df[["nutriscore_grade","grade_num"]].head(10)

In [26]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+8 XP — A4: Completed types and missing data handling (total=28)


## A5. GroupBy & Cross-tabulation (10 XP)

Summarize nutrients by high-level food groups and examine grade distributions.

### A5.1 — Grouped nutrient summary (5 XP)

For each high-level food group (`pnns_groups_1`), compute the **mean** and **standard deviation** of:  
`sugars_100g, fat_100g, salt_100g, proteins_100g`.  

💡 Hint: use [groupby] and [agg]

📖 Docs: [GroupBy.agg](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.agg.html)  

In [27]:
# A5.1 — Grouped nutrient summary by pnns_groups_1
metrics = {
    "sugars_100g": "median",
    "fat_100g": "median",
    "salt_100g": "median",
    "proteins_100g": "median"
}
grouped = df.groupby("pnns_groups_1").agg(metrics)
grouped = grouped.sort_index()
grouped.head(10)

### A5.2 — Find groups with lowest average sugars (2 XP)

Identify the **three food groups** with the **lowest average sugar content** (`sugars_100g`).  

💡 Hint: first compute the group means, then apply `.nsmallest(3)`.

📖 Docs: [Series.nsmallest](https://pandas.pydata.org/docs/reference/api/pandas.Series.nsmallest.html)  

In [28]:
# A5.2 — Groups with lowest average sugars
avg_sugar = df.groupby("pnns_groups_1")["sugars_100g"].mean().sort_values()
avg_sugar.head(10)

### A5.3 Crosstab of Nutri-Score by group (3 XP)

Let’s explore how nutrition **grades** distribute across food groups, as defined by the `pnns_groups_1` column (do not use the previous results from groupby, but work on the original dataframe).

**Task**
- Build a **crosstab** for counts of `nutriscore_grade` by `pnns_groups_1` for the **top 5 groups** (by count).  

💡 **What is a crosstab?**  
A *crosstabulation* (or contingency table) shows how two categorical variables are distributed against each other:  
- Rows = categories of one variable  
- Columns = categories of another variable  
- Cells = counts (or other aggregates) of their intersections  

**Why is this useful here?**  
In our dataset, a crosstab between `pnns_groups_1` (food group) and `nutriscore_grade` (nutrition label a–e) helps us see **how nutritional quality varies across food groups**.  
For example:  
- *Beverages* may cluster around `d` or `e`  
- *Fruits* may cluster around `a` or `b`  

This reveals patterns of nutritional quality across categories at a glance.
  
💡 Hint: use the pandas `crosstab` function.

📖 [pandas.crosstab](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html)

In [29]:
# A5.3 — Crosstab of Nutri-Score by group
ct = pd.crosstab(df["pnns_groups_1"], df["nutriscore_grade"]).loc[:, ["a","b","c","d","e"]]
ct.head(10)

In [30]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+10 XP — A5: Completed groupby and crosstab (total=38)


# Part B — Plots & Interpretation (25 XP)

## B1. Distributions (8 XP)

Explore sugar distributions overall and across groups.

### B1.1 — Histogram of sugars (3 XP)

Plot a **histogram** of `sugars_100g` to see how sugar values are distributed across all products.

💡 Hints:  
- Use [`plt.hist`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html).  
- Drop `na` values if still present on `sugar_100g`.
- Nutrient data often contains extreme outliers. To avoid a squashed plot, you can clip values above the 99th percentile:  
  ```python
  cutoff = df["sugars_100g"].quantile(0.99)
  data = df["sugars_100g"].clip(upper=cutoff)

In [31]:
# B1.1 — Histogram of sugars
sug = df["sugars_100g"].dropna()
plt.figure(figsize=(7,4))
plt.hist(sug, bins=50)
plt.xlabel("Sugars (g per 100g)")
plt.ylabel("Count")
plt.title("Distribution of sugars per 100g")
plt.show()

### B1.2 — Boxplot of sugars by food group (3 XP)

Compare sugar content across the **top 5 most frequent food groups** (`pnns_groups_1`).

💡 Hints:  
1. First, find the top 5 groups with `.value_counts().head(5)`. Consider removing the "unknown" group. 
2. Subset the DataFrame to only those groups.  
3. Create a **boxplot** with `plt.boxplot`, grouping values by food group.  
   - Set `showfliers=False` to hide extreme outliers for readability.  
   - Provide labels for each boxplot (the group names).  
4. Rotate x-axis labels if they overlap.  

📖 Docs: [matplotlib.pyplot.boxplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html)  

In [32]:
# B1.2 — Boxplot of sugars by top groups (exclude Unknown)
# Pick top 5 groups by count excluding 'Unknown'
grp_counts = df["pnns_groups_1"].value_counts()
top5 = [g for g in grp_counts.index if pd.notna(g) and g.lower() != "unknown"][:5]

subset = df[df["pnns_groups_1"].isin(top5)][["pnns_groups_1","sugars_100g"]].dropna()
plt.figure(figsize=(8,5))
subset.boxplot(column="sugars_100g", by="pnns_groups_1")
plt.title("Sugars by group (top 5)")
plt.suptitle("")
plt.xlabel("Group")
plt.ylabel("Sugars (g/100g)")
plt.show()

### B1.3 — Interpretation (2 XP)
Overall sugar distribution is **right-skewed** with many low-to-moderate sugar products and a tail of high-sugar items.  
Across groups, **Sugary snacks** and some **Beverages** tend to show higher medians and wider IQRs, while **Milk and dairy products** cluster lower, reflecting naturally lower sugar (or lactose-managed) products. Outliers likely correspond to desserts and sweetened drinks.


Complete:

- Right-skewed sugars: many low–moderate items, long high-sugar tail.
- Higher median & spread in Sugary snacks and some Beverages; Milk & dairy lower.
- Outliers mainly desserts and sweetened drinks.

In [33]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+8 XP — B1: Completed distribution visualization (total=46)


## B2 Nutri-Score trends (6 XP)

Analyze how **median nutrient values** evolve across **Nutri-Score grades**.

1. **Group by Nutri-Score grade**  
   - Use the ordered categorical `nutriscore_grade` defined earlier.  
   - Compute the **median** of the following columns:  
     - `sugars_100g`  
     - `fat_100g`  
     - `salt_100g`  
   - 📖 Docs: [DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

2. **Plot median values**  
   - Create a **single line plot** with one axis.  
   - Each nutrient should appear as a separate line, with markers for clarity.  
   - Suggested usage:  
     ```python
     medians.plot(
         kind="line",
         marker="o"
     )
     ```  
   - 📖 Docs: [DataFrame.plot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html)

**Hints**
- After grouping, use `.median()` to compute medians.  
- Ensure the `nutriscore_grade` column is categorical and ordered (A → E) to preserve logical axis ordering.  
- If grades are out of order, use:  
  ```python
  df["nutriscore_grade"] = pd.Categorical(
      df["nutriscore_grade"],
      categories=["a", "b", "c", "d", "e"],
      ordered=True
  )

In [34]:
# B2 — Nutri-Score trends (median nutrients by grade)
# Ensure ordered categories for plotting
df["nutriscore_grade"] = pd.Categorical(df["nutriscore_grade"], categories=["a","b","c","d","e"], ordered=True)

med = (df.groupby("nutriscore_grade")[["sugars_100g","fat_100g","salt_100g"]]
         .median())

# Line plot with markers for 3 nutrients
plt.figure(figsize=(8,5))
for col in med.columns:
    plt.plot(med.index.astype(str), med[col], marker="o", label=col)
plt.xlabel("Nutri-Score grade (a=best → e=worst)")
plt.ylabel("Median value (per 100g)")
plt.title("Median nutrients across Nutri-Score grades")
plt.legend()
plt.show()

med

In [35]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+8 XP — B2: Completed Nutri-Score Trends (total=54)


## B3. Scatter + rule-based flag (10 XP)

Examine the sugar–protein relationship and flag suspicious items.

**Tasks**
1. Make a **scatter plot** of `sugars_100g` (x) vs `proteins_100g` (y). Use small markers and some alpha for readability.
   - Docs: [matplotlib.pyplot.scatter](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html)
2. Create a boolean column `is_suspicious` defined by: `sugars_100g > 30` **and** `proteins_100g < 3`.
3. Show a table of the **top 5 suspicious products** with columns: `product_name, brands, pnns_groups_1, sugars_100g, proteins_100g` (sorted by high sugar then low protein).

**Goal**: mix visualization with simple logic to find extreme items.

In [36]:
# B3 — Scatter and suspicious flag
x = df["sugars_100g"]
y = df["proteins_100g"]

plt.figure(figsize=(6,5))
plt.scatter(x, y, s=12, alpha=0.4)
plt.xlabel("Sugars (g/100g)")
plt.ylabel("Proteins (g/100g)")
plt.title("Sugars vs Proteins")
plt.show()

df["is_suspicious"] = (df["sugars_100g"] > 30) & (df["proteins_100g"] < 3)

susp = df[df["is_suspicious"]].copy()
cols = ["product_name","brands","pnns_groups_1","sugars_100g","proteins_100g"]
susp = susp.sort_values(by=["sugars_100g","proteins_100g"], ascending=[False, True])
susp[cols].head(5)

In [37]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+10 XP — B3: Scatter plots and rule-based flags. (total=64)


# Part C (Extra): Brands & Categories (15 XP)

## C1. Brand averages (8 XP)

Compare **brands** by their **average Nutri-Score**.

1. **Prepare `brand_main`**  
   - Ensure you have a column `brand_main` (first brand, lowercased).  
   - You should already have it from **A3**. If not, create it as:  
     ```python
     df["brand_main"] = (
         df["brands"]
         .str.split(",")
         .str[0]
         .str.strip()
         .str.lower()
     )
     ```

2. **Numeric Nutri-Score mapping**  
   - Map grades to integers:  
     - `a → 0, b → 1, c → 2, d → 3, e → 4`  
   - Store the result in a new column `grade_num` with integer dtype.  
   - Handle **missing values** properly:  
     - Some entries may be `NaN` (actual missing values).  
     - Others may be the string `"nan"` — convert these to `NaN`.  
     - Remove rows with missing `brand_main` or `grade_num` before analysis.  
   - 📖 Docs: [Series.map](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html), [DataFrame.dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)

3. **Aggregate and visualize**  
   - Identify the **top 10 brands** by frequency:  
     ```python
     top_brands = df["brand_main"].value_counts().head(10).index
     ```  
   - Filter the dataset to keep only these brands.  
   - Compute the **mean `grade_num`** for each brand.  
   - Sort results ascending (best brands first).  
   - Plot as a **bar chart**.  
   - 📖 Docs:  
     - [Series.value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)  
     - [DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)  
     - [DataFrame.plot.bar](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html)  

**Hints**
- Use `replace("nan", np.nan)` before dropping NAs to catch string `"nan"`.  
- Use `df["grade_num"].astype("Int64")` for a nullable integer column.  
- Sorting ascending gives a **leaderboard** where **lower = healthier**.  

In [38]:
# C1 — Brand averages (mean grade_num among top brands)
# Ensure brand_main exists
if "brand_main" not in df.columns:
    df["brand_main"] = (
        df["brands"].str.split(",").str[0].str.strip().str.lower()
    )

# Clean and ensure grade_num is available
df["nutriscore_grade"] = df["nutriscore_grade"].astype(str).str.strip().str.lower().replace("nan", np.nan)
order = ["a","b","c","d","e"]
df["nutriscore_grade"] = pd.Categorical(df["nutriscore_grade"], categories=order, ordered=True)
grade_map = {"a":1,"b":2,"c":3,"d":4,"e":5}
df["grade_num"] = df["nutriscore_grade"].astype(str).map(grade_map).astype("Int64")

# Keep only top 10 brands by count
top_brands = df["brand_main"].value_counts().head(10).index
df_topb = df[df["brand_main"].isin(top_brands)].copy()

brand_avg = df_topb.groupby("brand_main")["grade_num"].mean().sort_values()
print(brand_avg)

# Bar chart
plt.figure(figsize=(8,4))
brand_avg.plot(kind="bar")
plt.ylabel("Mean grade (lower=better)")
plt.title("Top 10 brands by count: average Nutri-Score (1=a … 5=e)")
plt.show()

brand_avg.to_frame("mean_grade")

In [39]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+8 XP — C1: Completed Brand averages. (total=72)


## C2. Category leaderboard (8 XP)

Build a compact table for the top groups.

1. Take the **top 5** `pnns_groups_1` by count.
2. For those groups, compute: **count**, **mean sugars**, **mean salt**, and **% of grades in {a,b}**.
   - `% of {a,b}` can be computed using a boolean mask and `groupby(...).mean()*100`.
3. Show the resulting table sorted by `count` descending.

**Goal**: create a small multi-metric leaderboard per category.

In [49]:
# C2 — Category leaderboard
top5_groups = df["pnns_groups_1"].value_counts().head(5).index
dff = df[df["pnns_groups_1"].isin(top5_groups)].copy()

agg_tbl = dff.groupby("pnns_groups_1").agg(
    count=("pnns_groups_1","size"),
    mean_sugars=("sugars_100g","mean"),
    mean_salt=("salt_100g","mean")
)

# % of {a,b}
good_mask = dff["nutriscore_grade"].isin(["a","b"])
pct_good = (dff.assign(good=good_mask)
              .groupby("pnns_groups_1")["good"].mean()*100.0)

leader = agg_tbl.join(pct_good.rename("%_ab")).sort_values("count", ascending=False)
leader

In [40]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+8 XP — C2: Categoryy leaderboard. (total=80)


## Part D — PCA (optional, 20 XP)

### D1. PCA on nutrient features (20 XP)

Apply **Principal Component Analysis (PCA)** to reduce nutrient dimensions and interpret the main components.

1. **Select features**  
   - Nutrients: `sugars_100g`, `fat_100g`, `salt_100g`, `proteins_100g`, `energy-kcal_100g`, `fiber_100g`  
   - Drop rows with missing values before building the feature matrix.

2. **Standardize**  
   - Use `StandardScaler` to center and scale features.  
   - Standardization is important because nutrients are in different units (g, kcal, etc.).  
   - 📖 [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

3. **Fit PCA**  
   - Run `PCA()` on the standardized data.  
   - Plot both the explained variance ratio (per component) and the cumulative ratio.  
   - These show how much information each principal component captures.  
   - 📖 [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

4. **Loadings & PC1**  
   - Loadings indicate how strongly each feature contributes to a component.  
   - Identify the feature with the **largest absolute loading** in PC1 (the main driver of dietary variation).

5. **Correlation circle (PC1–PC2)**  
   - Draw a unit circle.  
   - Plot arrows from (0,0) to each feature’s coordinates on the first two PCs.  
   - Use loadings + eigenvalues from PCA to compute coordinates.  
   - Annotate arrows with variable names for clarity.

**Hints**
- Check `pca.explained_variance_ratio_` for variance plots.  
- Use `pca.components_.T` to access loadings.  
- A variable pointing near the PC1 or PC2 axis is strongly correlated with that component.  
- Opposite directions suggest negative correlation between nutrients.  


In [ ]:
# D1 — PCA on nutrient features
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 1) Select features
features = [c for c in ["sugars_100g","fat_100g","salt_100g","proteins_100g","energy-kcal_100g","fiber_100g"] if c in df.columns]
X = df[features].dropna().copy()

# 2) Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3) PCA
pca = PCA(n_components=2, random_state=0)
X_pca = pca.fit_transform(X_scaled)
expl = pca.explained_variance_ratio_
print("Explained variance ratio (PC1, PC2):", expl)

# 4) Loadings
loadings = pd.DataFrame(pca.components_.T, index=features, columns=["PC1","PC2"])
print("\nPCA loadings:")
print(loadings)

# Main driver (highest |loading| on PC1)
main_driver = loadings["PC1"].abs().sort_values(ascending=False).index[0]
print("\nMain driver of PC1:", main_driver)

# 5) Correlation circle (PC1–PC2)
theta = np.linspace(0, 2*np.pi, 200)
circle_x = np.cos(theta)
circle_y = np.sin(theta)

plt.figure(figsize=(6,6))
plt.plot(circle_x, circle_y)
plt.axhline(0); plt.axvline(0)
for var in features:
    x,y = loadings.loc[var, "PC1"], loadings.loc[var, "PC2"]
    plt.arrow(0,0,x,y, head_width=0.03, length_includes_head=True)
    plt.text(x*1.05, y*1.05, var, ha="center", va="center")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Correlation circle (PC1–PC2)")
plt.gca().set_aspect("equal", adjustable="box")
plt.show()

# Also return the PCA coordinates for reference
pca_df = pd.DataFrame(X_pca, columns=["PC1","PC2"]).join(X.reset_index(drop=True))
pca_df.head()

**Interpretation (D1).**  
PC1 typically contrasts **energy-dense, salty/fatty** items against **leaner / lower-salt** ones (high positive loadings on energy/fat/salt).  
PC2 often loads on **sugars vs proteins/fiber**, separating **sweet** foods from **protein-/fiber-richer** items. Exact directions follow the loadings table printed above.


- PC1: energy/fat/salt-dense ↔ lean/low-salt (largest |loadings| on energy, fat, salt).
- PC2: sugars ↔ proteins/fiber (sweet vs protein/fiber-rich foods).
- Driver check: confirm with the loadings table.


In [41]:
# Do not modify anything in this cell.
# Each task is worth a certain number of points.
# We call the award function with the appropriate number of points when you complete each task.

+20 XP — D: PCA completed. (total=100)


## Wrap-up & grade

In [42]:
grade_summary()

Final XP=100 → Grade=20/20


20